Ce bloc teste plusieurs variantes simples autour d’une stratégie “low-vol” sur un univers crypto (avec DOGE). On construit d’abord les rendements log et des features (volatilité rolling, momentum, trend filter sur BTC).

Ensuite on définit 4 stratégies : low-vol pur, low-vol avec filtre momentum, low-vol avec filtre trend BTC, et low-vol avec un vol targeting global. On backteste en long-only avec coûts (turnover * bps), puis on compare les perfs via un tableau et une courbe d’equity.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 0) params
SYMBOLS = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT", "XRPUSDT", "DOGEUSDT"]  # + doge
DATA_DIR = "binance_public_data"
START = "2018-01-01"
END   = "2025-12-31"

WIN_VOL = 20
WIN_MOM = 20
MA_TREND = 200

COST_BPS = 10
VOL_TARGET_ANN = 0.60
FREQ = 365


# 1) load (safe)
def load_symbol(sym):
    # 1a) file (ici tes fichiers sont 2021-2025)
    fn = f"{DATA_DIR}/{sym}_1d_2021_2025.csv"

    # 1b) check file
    if not os.path.exists(fn):
        print("skip missing:", sym)
        return None

    # 1c) read
    df = pd.read_csv(fn, parse_dates=["timestamp"], index_col="timestamp").sort_index()

    # 1d) dates
    df = df.loc[START:END]

    # 1e) returns log
    df["ret"] = np.log(df["close"] / df["close"].shift(1))

    return df.dropna()


# 2) load all
dfs = {}
symbols_ok = []

for s in SYMBOLS:
    df = load_symbol(s)
    if df is None or df.empty:
        continue
    dfs[s] = df
    symbols_ok.append(s)

print("symbols used:", symbols_ok)

if len(symbols_ok) < 2:
    raise ValueError("pas assez d'actifs pour backtest, check csv dispo")


# 3) align index
idx = None
for s in symbols_ok:
    idx = dfs[s].index if idx is None else idx.intersection(dfs[s].index)

# 4) returns matrix
rets = pd.DataFrame({s: dfs[s].loc[idx, "ret"] for s in symbols_ok}).dropna()

# 5) features
close = pd.DataFrame({s: dfs[s].loc[rets.index, "close"] for s in symbols_ok})

# 5a) vol rolling
vol = rets.rolling(WIN_VOL).std()

# 5b) momentum simple
mom = close.pct_change(WIN_MOM)


# 6) btc trend gate
btc_close = close["BTCUSDT"]
btc_ma = btc_close.rolling(MA_TREND).mean()
trend_on = (btc_close > btc_ma).astype(float)


# 7) baseline low-vol weights
q = vol.quantile(0.5, axis=1)
w_low = vol.le(q, axis=0).astype(float)


# 8) helper backtest
def run_bt(weights, rets, cost_bps=COST_BPS):
    # 8a) lag
    w = weights.shift(1).fillna(0.0)

    # 8b) long-only
    w = w.clip(lower=0.0)

    # 8c) normalise
    denom = w.sum(axis=1).replace(0, np.nan)
    w = w.div(denom, axis=0).fillna(0.0)

    # 8d) gross
    gross = (w * rets).sum(axis=1)

    # 8e) turnover + cost
    to = w.diff().abs().sum(axis=1).fillna(0.0)
    cost = to * (cost_bps / 10000.0)

    # 8f) net
    net = gross - cost
    return net, to, w


# 9) stats
def stats(r):
    r = r.dropna()
    eq = (1 + r).cumprod()

    mu = r.mean() * FREQ
    sig = r.std() * np.sqrt(FREQ)

    sharpe = mu / sig if sig > 0 else np.nan
    dd = (eq / eq.cummax() - 1.0).min()
    ann = eq.iloc[-1] ** (FREQ / len(r)) - 1 if len(r) > 0 else np.nan

    return float(ann), float(sig), float(sharpe), float(dd)


# 10) strategies
# A) low-vol
w_A = w_low.copy()

# B) low-vol + momentum
mom_ok = (mom > 0).astype(float)
w_B = w_low * mom_ok

# C) low-vol + btc trend
w_C = w_low.mul(trend_on, axis=0)

# D) low-vol + vol targeting (scale)
w_tmp = w_low.shift(1).fillna(0.0)
w_tmp = w_tmp.div(w_tmp.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)

port_ret_tmp = (w_tmp * rets).sum(axis=1)
port_vol = port_ret_tmp.rolling(60).std() * np.sqrt(FREQ)

scale = (VOL_TARGET_ANN / port_vol).clip(upper=1.0)
w_D = w_low.mul(scale, axis=0)


# 11) run
bt = {}
for name, w in [
    ("low_vol", w_A),
    ("low_vol_mom", w_B),
    ("low_vol_trend", w_C),
    ("low_vol_vtarget", w_D),
]:
    r, to, _ = run_bt(w, rets, cost_bps=COST_BPS)
    bt[name] = (r, to)


# 12) summary table
rows = []
for name, (r, to) in bt.items():
    ann, sig, sh, dd = stats(r)
    rows.append({
        "name": name,
        "ann_return": ann,
        "ann_vol": sig,
        "sharpe": sh,
        "max_dd": dd,
        "turnover_mean": float(to.mean()),
    })

summary = pd.DataFrame(rows).set_index("name").sort_values("sharpe", ascending=False)
print(summary)


# 13) plot equity
plt.figure(figsize=(12, 5))
for name, (r, _) in bt.items():
    eq = (1 + r.fillna(0)).cumprod()
    plt.plot(eq.index, eq, label=name)

plt.legend()
plt.title("Low-vol baseline vs simple overlays (net)")
plt.tight_layout()
plt.show()


symbols used: ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'SOLUSDT', 'XRPUSDT', 'DOGEUSDT']
                 ann_return   ann_vol    sharpe    max_dd  turnover_mean
name                                                                    
low_vol            0.102579  0.649064  0.481898 -0.816893       0.088950
low_vol_vtarget   -0.025993  0.625855  0.278719 -0.816893       0.087123
low_vol_trend      0.022382  0.391999  0.252888 -0.489617       0.080000
low_vol_mom       -0.039360  0.525792  0.207657 -0.834946       0.300091

![Image test 6](image/image_test_6.png)


La stratégie low-vol de base est celle qui présente le meilleur compromis rendement / risque sur la période considérée. Elle affiche un rendement annualisé positif (≈10 %) avec un Sharpe proche de 0.48, ce qui suggère que la sélection des actifs les moins volatils au sein de l’univers crypto permet déjà de lisser une partie du risque sans trop sacrifier la performance. En revanche, le drawdown reste très élevé (≈ −82 %), ce qui rappelle que même une approche low-vol reste fortement exposée aux phases de stress du marché crypto.

La stratégie low-vol avec vol targeting réduit légèrement la volatilité annualisée, mais au prix d’une performance négative. Cela indique que le scaling global de l’exposition intervient trop souvent dans des phases où le marché repart à la hausse, ce qui dégrade le rendement sans offrir une réelle protection supplémentaire sur le drawdown, qui reste identique à la stratégie low-vol de base.

La stratégie low-vol avec filtre de tendance sur Bitcoin est nettement plus défensive. Elle réduit fortement la volatilité et surtout le drawdown maximal (≈ −49 %), ce qui constitue une amélioration importante en termes de contrôle du risque. En contrepartie, le rendement est faible et le Sharpe reste modeste. Cette stratégie illustre bien le trade-off classique entre protection du capital et participation aux phases haussières.

Enfin, la stratégie low-vol avec filtre momentum est la moins convaincante. Le rendement est négatif et le turnover est très élevé, ce qui suggère que le signal de momentum à court terme (20 jours) est trop instable dans l’univers crypto. Les changements fréquents de positions génèrent des coûts qui détériorent fortement la performance.

Globalement, ces résultats montrent que des overlays simples peuvent améliorer certains aspects du profil de risque (notamment le drawdown via le filtre de tendance), mais qu’ils doivent être soigneusement calibrés. La stratégie low-vol constitue une base solide, tandis que les filtres additionnels nécessitent une optimisation plus fine (fenêtres, seuils, fréquence) pour réellement créer de la valeur.